In [1]:
import numpy as np
import pandas as pd
import scipy as sp

In [2]:
from tqdm import tqdm_notebook


In [3]:
from nltk.corpus import movie_reviews
import nltk
nltk.download('movie_reviews')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/alexey/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


True

In [4]:
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

In [5]:
negfeats = [movie_reviews.words(fileids=[f]) for f in negids]
posfeats = [movie_reviews.words(fileids=[f]) for f in posids]

In [7]:
len(negfeats)

The history saving thread hit an unexpected error (OperationalError('unable to open database file',)).History will not be written to the database.

Exception in thread IPythonHistorySavingThread:
Traceback (most recent call last):
  File "/home/alexey/.virtualenvs/yandex/lib/python3.6/site-packages/IPython/core/history.py", line 834, in run
  File "<decorator-gen-23>", line 2, in writeout_cache
  File "/home/alexey/.virtualenvs/yandex/lib/python3.6/site-packages/IPython/core/history.py", line 58, in needs_sqlite
  File "/home/alexey/.virtualenvs/yandex/lib/python3.6/site-packages/IPython/core/history.py", line 780, in writeout_cache
  File "/home/alexey/.virtualenvs/yandex/lib/python3.6/site-packages/IPython/core/history.py", line 764, in _writeout_input_cache
sqlite3.OperationalError: unable to open database file

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/alexey/.virtualenvs/yandex/lib/python3.6/site-packages/ipykernel/iostream.py", line 97, in _event_pipe
AttributeError: '_thread._local' object has no attribute 'event_pipe'

During handling of the above 

1000

In [8]:
len(posfeats)

1000

In [59]:
all_reviews = posfeats + negfeats
all_classes = [1]*len(negfeats) + [0]*len(posfeats)

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [136]:
vector = CountVectorizer(input='content')

In [138]:
reviews_better = [' '.join(l) for l in all_reviews]

In [139]:
transformed = vector.fit_transform(reviews_better)

In [127]:
transformed.shape

(2000, 39659)

In [69]:
_, features = transformed.shape
with open('3.txt', 'w') as f:
    f.write('{}'.format(features))

In [77]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [78]:
pipe = Pipeline([('vectorizer', CountVectorizer()), ('lr', LogisticRegression())])

In [79]:
acc_cv = cross_val_score(X=reviews_better, y=all_classes, estimator=pipe, n_jobs=-1, scoring='accuracy')

In [115]:
with open('4.txt', 'w') as f:
    f.write(str(np.mean(acc_cv)))

In [81]:
auc_cv = cross_val_score(X=reviews_better, y=all_classes, estimator=pipe, n_jobs=-1, scoring='roc_auc')

In [116]:
with open('5.txt', 'w') as f:
    f.write(str(np.mean(auc_cv)))

In [128]:
log_reg = LogisticRegression()
log_reg.fit(transformed, all_classes)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [89]:
pipe.fit(X=reviews_better, y=all_classes)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [133]:
best_features = np.argsort(np.abs(log_reg.coef_[0]))

In [134]:
log_reg.coef_[0][best_features]

array([-8.41340984e-08, -8.41340984e-08, -8.41340984e-08, ...,
       -5.92901730e-01, -6.36618357e-01, -7.82176024e-01])

In [140]:
np.array(vector.get_feature_names())[best_features]

array(['schubert', 'stepahne', 'juxtapose', ..., 'worst', 'unfortunately',
       'bad'], dtype='<U58')

In [141]:
coeffs = log_reg.coef_[0]
[vector.get_feature_names()[list(coeffs).index(i)] for i in sorted(coeffs)[:5]]

['bad', 'unfortunately', 'worst', 'waste', 'nothing']